## Saâd AZIZ ALAOUI & Simoné IANNELLI

In [2]:
import numpy as np
from numpy.random import normal
from scipy.stats import weibull_min, logistic, norm, gamma, expon, truncweibull_min
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [3]:
# Vecteur des failure times (les individus censurés ont un nan)
t =np.array([12,17,21, 25, 11,  26,  27,  30, 13, 12,  21,20,  23,  25,  23,  29, 35, np.nan, 31, 36,
     32, 27,  23, 12, 18, np.nan, np.nan,  38, 29, 30, np.nan, 32, np.nan, np.nan, np.nan, np.nan, 25,  30, 37, 27,
     22, 26, np.nan, 28, 19,  15,  12,  35, 35, 10,  22, 18, np.nan,  12, np.nan, np.nan, 31,  24, 37, 29,
     27, 18,  22, 13, 18,  29,  28, np.nan, 16, 22,  26, 19, np.nan, np.nan,  17,  28, 26,  12, 17, 26])
# les individus censurés ont une valeur différente de zéro (leur temps de censure en semaines)
tcensor =np.array([0, 0,  0, 0, 0,  0,  0,  0, 0, 0,  0, 0,  0,  0,  0,  0, 0, 40, 0, 0,
     0, 0,  0, 0, 0, 40, 40,  0, 0, 0, 40, 0, 40, 40, 40, 40, 0,  0, 0, 0,
     0, 0, 10, 0, 0,  0,  0,  0, 0, 0,  0, 0, 24,  0, 40, 40, 0,  0, 0, 0,
     0, 0,  0, 0, 0,  0,  0, 20, 0, 0,  0, 0, 29, 10,  0,  0, 0,  0, 0, 0])

is_censored = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 
0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 
0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 
0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0])

t_uncensored =np.array([12, 17, 21, 25, 11, 26, 27, 30, 13, 12, 21, 20, 23, 25, 23, 
29, 35, 31, 36, 32, 27, 23, 12, 18, 38, 29, 30, 32, 25, 30, 37, 
27, 22, 26, 28, 19, 15, 12, 35, 35, 10, 22, 18, 12, 31, 24, 37, 
29, 27, 18, 22, 13, 18, 29, 28, 16, 22, 26, 19, 17, 28, 26, 12, 
17, 26])

group_mice = np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 
2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 
3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 
4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])

len_uncens = len(t_uncensored)
len_cens = 80 - len_uncens
len_group = 4

In [6]:
init_r = 0 
init_beta1 = 0
init_beta2 = 0
init_beta3 = 0
init_beta4 = 0
init_vehcontrol = 0
init_testsub = 0
init_poscontrol = 0

65

In [13]:
weibull_min.rvs(2.4, loc=0, scale=5)

6.1203189501240205

In [7]:
t_1 = t[:20]
tcensor_1 = tcensor[:20]
is_censored_1 = is_censored[:20]
uncensored_array_1 = [t_1[i] for i in range(len(t_1)) if is_censored_1[i] == 0]
censored_array_1 = [tcensor_1[i] for i in range(len(tcensor_1)) if is_censored_1[i] == 1]
censored_array_1

[40]

In [ ]:
def mice(nchain,init_r,init_beta1,init_beta2,init_beta3,init_beta4,init_vehcontrol,init_testsub,init_poscontrol,prop_sd,t,t_cen) :

    ## nchain est la taille de la chaine 
    ## init_alpha, init_beta1, init_beta2,init_beta3,init_beta4,init_vehcontrol,init_testsub,init_poscontrol sont les val init de nos param
    ## prop_sd est la variance pour la MC
    ## la fonction résoud le probleme WINBUGS MICE MCMC within GIBBS.

    r = init_r
    beta1 = init_beta1
    beta2 = init_beta2
    beta3 = init_beta3
    beta4 = init_beta4
    veh_control = init_vehcontrol
    testsub = init_testsub
    poscontrol = init_poscontrol	


    chain = np.zeros((nchain+1, 8))
    acc_rate = np.zeros(5)
    chain[0,0] = init_r
    chain[0,1] = init_beta1
    chain[0,2] = init_beta2
    chain[0,3] = init_beta3
    chain[0,4] = init_beta4
    chain[0,5] = init_vehcontrol
    chain[0,6] = init_testsub
    chain[0,7] = init_poscontrol

    t_1 = t[:20]
    tcensor_1 = tcensor[:20]
    is_censored_1 = is_censored[:20]
    uncensored_array_1 = [t_1[i] for i in range(len(t_1)) if is_censored_1[i] == 0]
    censored_array_1 = [tcensor_2[i] for i in range(len(tcensor_2)) if is_censored_1[i] == 1]

    t_2 = t[20:40]
    tcensor_2 = tcensor[20:40]
    is_censored_2 = is_censored[20:40]
    uncensored_array_2 = [t_2[i] for i in range(len(t_2)) if is_censored_2[i] == 0]
    censored_array_2 = [tcensor_2[i] for i in range(len(tcensor_2)) if is_censored_2[i] == 1]

    t_3= t[40:60]
    tcensor_3 = tcensor[40:60]
    is_censored_3 = is_censored[40:60]
    uncensored_array_3 = [t_3[i] for i in range(len(t_3)) if is_censored_3[i] == 0]
    censored_array_3 = [tcensor_3[i] for i in range(len(tcensor_3)) if is_censored_3[i] == 1]

    t_4 = t[60:80]
    tcensor_4 = tcensor[60:80]
    is_censored_4 = is_censored[60:80]
    uncensored_array_4 = [t_4[i] for i in range(len(t_4)) if is_censored_4[i] == 0]
    censored_array_4 = [tcensor_4[i] for i in range(len(tcensor_4)) if is_censored_4[i] == 1]
    
    for i in range(nchain) : 

        # Mise à jour de r pour commencer 

        prop = normal(loc = r, scale = prop_sd[0])

        top_1 = (-10**-3)*prop + sum( (np.exp(beta1)-1) * np.log(uncensored_array_1/prop)) - sum( (np.exp(beta1)) * (uncensored_array_1/prop)) + sum( (np.exp(beta1)-1) * np.log(censored_array_1/prop))
        top_2 = (-10**-3)*prop + sum( (np.exp(beta2)-1) * np.log(uncensored_array_2/prop)) - sum( (np.exp(beta2)) * (uncensored_array_2/prop)) + sum( (np.exp(beta2)-1) * np.log(censored_array_2/prop))
        top_3 = (-10**-3)*prop + sum( (np.exp(beta3)-1) * np.log(uncensored_array_3/prop)) - sum( (np.exp(beta3)) * (uncensored_array_3/prop)) + sum( (np.exp(beta3)-1) * np.log(censored_array_3/prop))
        top_4 = (-10**-3)*prop + sum( (np.exp(beta4)-1) * np.log(uncensored_array_4/prop)) - sum( (np.exp(beta4)) * (uncensored_array_4/prop)) + sum( (np.exp(beta4)-1) * np.log(censored_array_4/prop))

        top = top_1 + top_2 + top_3 + top_4

        bottom_1 = (-10**-3)*r + sum( (np.exp(beta1)-1) * np.log(uncensored_array_1/r)) - sum( (np.exp(beta1)) * (uncensored_array_1/r)) + sum( (np.exp(beta1)-1) * np.log(censored_array_1/r))
        bottom_2 = (-10**-3)*r + sum( (np.exp(beta2)-1) * np.log(uncensored_array_2/r)) - sum( (np.exp(beta2)) * (uncensored_array_2/r)) + sum( (np.exp(beta2)-1) * np.log(censored_array_2/r))
        bottom_3 = (-10**-3)*r + sum( (np.exp(beta3)-1) * np.log(uncensored_array_3/r)) - sum( (np.exp(beta3)) * (uncensored_array_3/r)) + sum( (np.exp(beta3)-1) * np.log(censored_array_3/r))
        bottom_4 = (-10**-3)*r + sum( (np.exp(beta4)-1) * np.log(uncensored_array_4/r)) - sum( (np.exp(beta4)) * (uncensored_array_4/r)) + sum( (np.exp(beta4)-1) * np.log(censored_array_4/r))

        bottom = bottom_1 + bottom_2 + bottom_3 + bottom_4

        acc_prob = np.exp(top - bottom)

        if np.random.uniform() < acc_prob:
            r = prop
            acc_rate[0] += 1

        # Mise à jour de Beta 1 
        prop = normal(loc = beta1, scale = prop_sd[1])

        top = - (prop**2 / (2 * 1e3)) + sum( (np.exp(prop)-1) * np.log(uncensored_array_1/r)) - sum( (np.exp(prop)) * (uncensored_array_1/r)) + sum( (np.exp(prop)-1) * np.log(censored_array_1/r))
        bottom = - (beta1**2 / (2 * 1e3)) + sum( (np.exp(beta1)-1) * np.log(uncensored_array_1/r)) - sum( (np.exp(beta1)) * (uncensored_array_1/r)) + sum( (np.exp(beta1)-1) * np.log(censored_array_1/r))
        acc_prob = np.exp(top - bottom)

        if np.random.uniform() < acc_prob:
            beta1 = prop
            acc_rate[1] += 1

        median1 = (np.log(2) * np.exp(-beta1)) ** 1/r

        # Mise à jour de Beta 2
        prop = normal(loc = beta2, scale = prop_sd[2])

        top = - (prop**2 / (2 * 1e3)) + sum( (np.exp(prop)-1) * np.log(uncensored_array_2/r)) - sum( (np.exp(prop)) * (uncensored_array_2/r)) + sum( (np.exp(prop)-1) * np.log(censored_array_2/r))
        bottom = - (beta2**2 / (2 * 1e3)) + sum( (np.exp(beta2)-1) * np.log(uncensored_array_1/r)) - sum( (np.exp(beta2)) * (uncensored_array_1/r)) + sum( (np.exp(beta2)-1) * np.log(censored_array_1/r))
        acc_prob = np.exp(top - bottom)

        if np.random.uniform() < acc_prob:
            beta2 = prop
            acc_rate[2] += 1

        median2 = (np.log(2) * np.exp(-beta2)) ** 1/r




    return(chain,acc_rate)

